In [2]:
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from tqdm import tqdm
from utils.utils import get_readable_file_size

In [3]:
ehr_data_dir = "/home/mengliang/DatasetFolder/mimiciv/2.2"

## Load MIMIC CXR dataset

MIMIC CXR dataset merged several tables from CXR dataset, including disease, diagnosis report.

The patient number: 65379  
The study number: 227827  
The image number: 377095

In [18]:
ehr_cxr_path = os.path.join("database", "mimic_iv_merge.csv")
df_ehr_cxr = pd.read_csv(ehr_cxr_path, index_col=False)
df_ehr_cxr.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,...,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,path,split,report_path
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train,files/p10/p10000032/s50414267.txt
1,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,train,files/p10/p10000032/s50414267.txt
2,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train,files/p10/p10000032/s53189527.txt
3,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,files/p10/p10000032/s53189527/e084de3b-be89b11...,train,files/p10/p10000032/s53189527.txt
4,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train,files/p10/p10000032/s53911762.txt


In [19]:
print(df_ehr_cxr.columns)
print(df_ehr_cxr["ViewPosition"].value_counts())

print("Table length: ", len(df_ehr_cxr))
print("Patients: ", len(df_ehr_cxr["subject_id"].unique()))
print("CXR times:", len(df_ehr_cxr["study_id"].unique()))

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows',
       'Columns', 'StudyDate', 'StudyTime',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'path', 'split',
       'report_path'],
      dtype='object')
ViewPosition
AP                147169
PA                 96155
LATERAL            82852
LL                 35129
PA LLD                 4
LAO                    3
RAO                    3
AP AXIAL               2
AP LLD                 2
XTABLE LATERAL         2
AP RLD                 2
SWIMMERS               1
PA RLD                 1
LPO                    1
Name: count, dtype: int64
Table length:

Since the CXR images are not provided the admission id, we need to find the corresponding hadm_id based on the date and time of the CXR image. Before this, we need convert the date and time to certain format.

In [20]:
selected_column = ['subject_id', 'study_id', 'dicom_id', 'PerformedProcedureStepDescription', 
                   'ViewPosition', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 
                   'ViewCodeSequence_CodeMeaning', 'path', 'report_path']

df_cxr_selected_column = df_ehr_cxr[selected_column]

df_cxr_selected_column.loc[:,'StudyDate'] = pd.to_datetime(df_cxr_selected_column.loc[:,'StudyDate'].astype(str), format='%Y%m%d')
# convert study date to string format
df_cxr_selected_column.loc[:,'StudyDate'] = df_cxr_selected_column.loc[:,'StudyDate'].dt.strftime('%Y-%m-%d')
# convert study time to string format
df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:,'StudyTime'].astype(str).str.split('.').str[0].str.zfill(6), format='%H%M%S')
df_cxr_selected_column.loc[:,'StudyTime'] = df_cxr_selected_column.loc[:,'StudyTime'].dt.strftime('%H:%M:%S')
# merge study date and study time

df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:, 'StudyDate'].dt.strftime('%Y-%m-%d') + ' ' + df_cxr_selected_column.loc[:, 'StudyTime'].dt.strftime('%H:%M:%S'))
# convert study time to string format
df_cxr_selected_column.head()

,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,3056,2544,2180-05-06,2180-05-06 21:30:14,postero-anterior,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt
1,10000032,50414267,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,CHEST (PA AND LAT),LATERAL,3056,2544,2180-05-06,2180-05-06 21:30:14,lateral,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,files/p10/p10000032/s50414267.txt
2,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,CHEST (PA AND LAT),PA,3056,2544,2180-06-26,2180-06-26 16:55:00,postero-anterior,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt
3,10000032,53189527,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,CHEST (PA AND LAT),LATERAL,3056,2544,2180-06-26,2180-06-26 16:55:00,lateral,files/p10/p10000032/s53189527/e084de3b-be89b11...,files/p10/p10000032/s53189527.txt
4,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,CHEST (PORTABLE AP),AP,2705,2539,2180-07-23,2180-07-23 08:05:56,antero-posterior,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt


## Load admissions dataset

Admissions dataset provides the information about the admission of patients to the hospital. It contains information about the admission id, patient id, admission type, discharge disposition, admission time, discharge time, length of stay, etc.

In admissions table:


In [7]:
ehr_admissions = pd.read_csv(os.path.join(ehr_data_dir, "hosp/admissions.csv.gz"), compression="gzip")
ehr_admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [21]:
print("Patients: ", len(ehr_admissions["subject_id"].unique()))
print("Admissions: ", len(ehr_admissions["hadm_id"].unique()))
print("Table length: ", len(ehr_admissions))

Patients:  180733
Admissions:  431231
Table length:  431231


search the corresponding hadm_id based on the cxr image date and time.

In [22]:
# convert time to datetime format
ehr_admissions.loc[:, 'admittime'] = pd.to_datetime(ehr_admissions.loc[:, 'admittime'])
ehr_admissions.loc[:, 'dischtime'] = pd.to_datetime(ehr_admissions.loc[:, 'dischtime'])
df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:, 'StudyTime'])

# process by subject_id
result_list = []

# enumerate all subject_id
for subject_id, group_B in df_cxr_selected_column.groupby('subject_id'):
    # obtain corresponding admission record
    group_A = ehr_admissions[ehr_admissions['subject_id'] == subject_id]

    # filter records within the admission time range
    matched = group_B.apply(
        lambda row: group_A.loc[
            (group_A['admittime'] <= row['StudyTime']) & 
            (group_A['dischtime'] >= row['StudyTime']), 
            'hadm_id'
        ].values, 
        axis=1
    )

    # record the matched records
    for index, hadm_ids in enumerate(matched):
        if len(hadm_ids) > 0:
            for hadm_id in hadm_ids:
                result_list.append({
                    'subject_id': subject_id,
                    'hadm_id': hadm_id,
                    #'study_id': group_B.iloc[index]['study_id'],
                    'StudyTime': group_B.iloc[index]['StudyTime'],
                })

# convert to DataFrame
result_df = pd.DataFrame(result_list)
result_df.head()

,subject_id,hadm_id,StudyTime
0,10000032,25742920,2180-08-05 23:44:24
1,10000764,27897940,2132-10-15 08:40:47
2,10000764,27897940,2132-10-15 08:40:47
3,10000764,27897940,2132-10-15 08:40:47
4,10000935,25849114,2187-10-16 12:39:45


In [23]:
result_df.drop_duplicates(inplace=True, keep="first")
print(len(result_df))
print(len(result_df["subject_id"].unique()))
print(len(result_df["hadm_id"].unique()))

115002
26486
40191


In [24]:
result_df.to_csv('outputs/ehr_cxr_subject.csv', index=False)

In [25]:
df_ehr_cxr_cross = pd.merge(df_cxr_selected_column, result_df, how = 
                      'inner', on=['subject_id', 'StudyTime'])
df_ehr_cxr_cross.drop_duplicates(inplace=True, keep='first')
df_ehr_cxr_cross.head()

,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path,hadm_id
0,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,CHEST (PORTABLE AP),AP,3056,2544,2180-08-05,2180-08-05 23:44:24,antero-posterior,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,files/p10/p10000032/s56699142.txt,25742920
1,10000764,57375967,096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4,CHEST (PA AND LAT),AP,2544,3056,2132-10-15,2132-10-15 08:40:47,antero-posterior,files/p10/p10000764/s57375967/096052b7-d256dc4...,files/p10/p10000764/s57375967.txt,27897940
2,10000764,57375967,b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f,CHEST (PA AND LAT),LATERAL,3056,2544,2132-10-15,2132-10-15 08:40:47,lateral,files/p10/p10000764/s57375967/b79e55c3-735ce5a...,files/p10/p10000764/s57375967.txt,27897940
3,10000764,57375967,dcfeeac4-1597e318-d0e6736a-8b2c2238-47ac3f1b,CHEST (PA AND LAT),LATERAL,3056,2544,2132-10-15,2132-10-15 08:40:47,lateral,files/p10/p10000764/s57375967/dcfeeac4-1597e31...,files/p10/p10000764/s57375967.txt,27897940
4,10000935,50578979,d0b71acc-b5a62046-bbb5f6b8-7b173b85-65cdf738,CHEST (PORTABLE AP),AP,2870,2402,2187-10-16,2187-10-16 12:39:45,antero-posterior,files/p10/p10000935/s50578979/d0b71acc-b5a6204...,files/p10/p10000935/s50578979.txt,25849114


In [27]:
df_ehr_cxr_cross.drop_duplicates(inplace=True, keep="first", subset=["dicom_id"])
print(len(df_ehr_cxr_cross["subject_id"].unique()))
print(len(df_ehr_cxr_cross["study_id"].unique()))
print(len(df_ehr_cxr_cross["hadm_id"].unique()))
print(len(df_ehr_cxr_cross["dicom_id"].unique()))
print(len(df_ehr_cxr_cross))

26486
110915
40190
146369
146369


## Load ICU stays

In [28]:
icu_icustays_path = os.path.join(ehr_data_dir, "icu/icustays.csv.gz")
df_icu_icustays = pd.read_csv(icu_icustays_path, index_col=False, compression="gzip")
df_icu_icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588


In [29]:
selected_column = ["subject_id", "hadm_id"]
icu_subject_hadm_df = df_icu_icustays[selected_column]
icu_subject_hadm_df.head()

,subject_id,hadm_id
0,10000032,29079034
1,10000980,26913865
2,10001217,24597018
3,10001217,27703517
4,10001725,25563031


In [30]:
print(len(icu_subject_hadm_df))
print(len(icu_subject_hadm_df["subject_id"].unique()))
print(len(icu_subject_hadm_df["hadm_id"].unique()))
print(len(df_icu_icustays["stay_id"].unique()))

73181
50920
66239
73181


In [31]:
icu_admissions = pd.read_csv(os.path.join("outputs", "icu_admissions.csv.gz"), compression="gzip")
icu_admissions.head()

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,ew emer.,emergency room,home,medicaid,widowed,white
1,10000980,26913865,2189-06-27 07:38:00,2189-07-03 03:00:00,ew emer.,emergency room,home health care,medicare,married,black/african american
2,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,ew emer.,emergency room,home health care,other,married,white
3,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,direct emer.,physician referral,home health care,other,married,white
4,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,ew emer.,pacu,home,other,married,white


In [32]:
print(len(icu_admissions["subject_id"].unique()))

50920


In [33]:
# convert time to datetime format
icu_admissions.loc[:, 'admittime'] = pd.to_datetime(icu_admissions.loc[:, 'admittime'])
icu_admissions.loc[:, 'dischtime'] = pd.to_datetime(icu_admissions.loc[:, 'dischtime'])
df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:, 'StudyTime'])

# process by subject_id
result_list = []

# enumerate all subject_id
for subject_id, group_B in df_cxr_selected_column.groupby('subject_id'):
    # obtain corresponding admission record
    group_A = icu_admissions[icu_admissions['subject_id'] == subject_id]

    # filter records within the admission time range
    matched = group_B.apply(
        lambda row: group_A.loc[
            (group_A['admittime'] <= row['StudyTime']) & 
            (group_A['dischtime'] >= row['StudyTime']), 
            'hadm_id'
        ].values, 
        axis=1
    )

    # record the matched records
    for index, hadm_ids in enumerate(matched):
        if len(hadm_ids) > 0:
            for hadm_id in hadm_ids:
                result_list.append({
                    'subject_id': subject_id,
                    'hadm_id': hadm_id,
                    #'study_id': group_B.iloc[index]['study_id'],
                    'StudyTime': group_B.iloc[index]['StudyTime'],
                })

# convert to DataFrame
result_df = pd.DataFrame(result_list)
result_df.head()

,subject_id,hadm_id,StudyTime
0,10001217,27703517,2157-12-18 19:16:10
1,10001884,26184834,2131-01-15 04:45:09
2,10001884,26184834,2131-01-12 04:56:56
3,10001884,26184834,2131-01-13 04:49:18
4,10001884,26184834,2131-01-10 12:54:30


In [34]:
result_df.drop_duplicates(inplace=True, keep="first")
print(len(result_df))
print(len(result_df["subject_id"].unique()))
print(len(result_df["hadm_id"].unique()))

75686
13170
16268


In [35]:
result_df.to_csv('outputs/icu_cxr_subject_info.csv', index=False)

In [36]:
df_icu_cxr = pd.merge(df_cxr_selected_column, result_df, how = 
                      'inner', on=['subject_id', 'StudyTime'])
df_icu_cxr.drop_duplicates(inplace=True, keep='first')
df_icu_cxr.head()

,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path,hadm_id
0,10001217,58913004,5e54fc9c-37c49834-9ac3b915-55811712-9d959d26,CHEST (PORTABLE AP),AP,3050,2539,2157-12-18,2157-12-18 19:16:10,antero-posterior,files/p10/p10001217/s58913004/5e54fc9c-37c4983...,files/p10/p10001217/s58913004.txt,27703517
1,10001884,50376803,469d0d94-3dad5068-efac76ef-a28cc502-68fe6275,CHEST (PORTABLE AP),AP,2539,2517,2131-01-15,2131-01-15 04:45:09,antero-posterior,files/p10/p10001884/s50376803/469d0d94-3dad506...,files/p10/p10001884/s50376803.txt,26184834
2,10001884,50712381,7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9,CHEST (PORTABLE AP),AP,2287,2532,2131-01-12,2131-01-12 04:56:56,antero-posterior,files/p10/p10001884/s50712381/7b25b3ed-e780a52...,files/p10/p10001884/s50712381.txt,26184834
3,10001884,56722923,c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f,CHEST (PORTABLE AP),AP,2539,3050,2131-01-13,2131-01-13 04:49:18,antero-posterior,files/p10/p10001884/s56722923/c1ad3e27-62d05ef...,files/p10/p10001884/s56722923.txt,26184834
4,10001884,57156853,9fd47edd-07087209-b901811e-3e9e5f50-f382f611,CHEST (PORTABLE AP),AP,2964,2539,2131-01-10,2131-01-10 12:54:30,antero-posterior,files/p10/p10001884/s57156853/9fd47edd-0708720...,files/p10/p10001884/s57156853.txt,26184834


In [38]:
print("Patients: ", len(df_icu_cxr["subject_id"].unique()))
print("CXR times: ", len(df_icu_cxr["study_id"].unique()))
print("Admissions: ", len(df_icu_cxr["hadm_id"].unique()))
print("Images: ", len(df_icu_cxr["dicom_id"].unique()))
print("Table length: ", len(df_icu_cxr))

Patients:  13170
CXR times:  73897
Admissions:  16268
Images:  89483
Table length:  89483
